In [346]:
from importlib import reload # python 2.7 does not require this
from yahoofinancials import YahooFinancials
import numpy as np
import json as js

In [480]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    ish = []
    cfsh = []
    summaryData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_incomeStatementHistory(self):
        self.ish = self.get_financial_stmts('annual', 'income')['incomeStatementHistory']
        return self.ish
    
    def get_cashflowStatementHistory(self):
        self.cfsh = self.get_financial_stmts('annual','cash')['cashflowStatementHistory']
        return self.cfsh
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            if not(self.bshData[self.ticker][i][date_key]):    
                break
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
    
    def get_ROIC(self, numofYears=20):
        roic = []
        if not(self.cfsh):
            self.get_cashflowStatementHistory()
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            if not(self.bshData[self.ticker][i][date_key]):    
                break
            equity = self.bshData[self.ticker][i][date_key]['totalStockholderEquity']
            if self.bshData[self.ticker][i][date_key].get('shortLongTermDebt') is None or not(self.bshData[self.ticker][i][date_key]['shortLongTermDebt']):
                debt_short = 0
            else:
                debt_short = self.bshData[self.ticker][i][date_key].get('shortLongTermDebt')
            if self.bshData[self.ticker][i][date_key].get('longTermDebt') is None or not(self.bshData[self.ticker][i][date_key]['longTermDebt']) :
                debt_long = 0
            else:
                debt_long = self.bshData[self.ticker][i][date_key]['longTermDebt']
            debt = debt_short + debt_long
            date_key = list(self.cfsh[self.ticker][i].keys())[0]
            if not(self.cfsh[self.ticker][i][date_key]):    
                break
            netincome = self.cfsh[self.ticker][i][date_key]['netIncome']
            roic_year = netincome/(equity + debt)
            roic.append(roic_year)
        return roic 
    
    def get_totalCashFromOperatingActivities(self, numofYears=20):
        totalCash = []
        if not(self.cfsh):
            self.get_cashflowStatementHistory()        
        for i in range(min(np.size(self.cfsh[self.ticker]), numofYears)):
            date_key = list(self.cfsh[self.ticker][i].keys())[0]
            if not(self.cfsh[self.ticker][i][date_key]):    
                break
            totalCash.append(self.cfsh[self.ticker][i][date_key]['totalCashFromOperatingActivities'])  
        return totalCash
    
    def get_pricetoSales(self):
        if not(self.summaryData):
            self.summaryData = self.get_summary_data()
        if not(self.summaryData[self.ticker]):
            return 'na'
        return self.summaryData[self.ticker]['priceToSalesTrailing12Months']
    
    def get_marketCap_B(self):
        if not(self.summaryData):
            self.summaryData = self.get_summary_data()
        if not(self.summaryData[self.ticker]):
            return 'na'
        return self.summaryData[self.ticker]['marketCap']/1000000000
    
    def get_CF_GR_median(self, totalCash):
        gr = []
        for v in range(np.size(totalCash)-1):
            gr.append((totalCash[v]-totalCash[v+1])/abs(totalCash[v+1]))
        #print(gr)
        return np.size(totalCash)-1, np.median(gr) 
    
    #use mean of each year    
    def get_BV_GR_median(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.median(gr)
    
    #use mean of each year    
    def get_ROIC_median(self, roic):
        return np.size(roic), np.median(roic)
    
    def get_BV_GR_max(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.max(gr)
    
    def growthRate(self, cur,init, years):
        if cur <=0 or init<=0:
            return -1
        return (cur/init)**(1/years)-1
    
    def get_BV_GR_mean(self, bv):
        gr = []
        BV_GR = self.growthRate(bv[0], bv[np.size(bv)-1], np.size(bv)-1)
        if BV_GR==-1:
            for v in range(np.size(bv)-1):
                gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
            BV_GR = np.mean(gr)
        return np.size(bv)-1, BV_GR
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        if not(cEPS) or not(growth) or not(PE):
            return 'NA'
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def payBackTime(self, price, cEPS, growth):
        tmp = 0
        i = 0
        if cEPS < 0:
            return 0
        while(growth>0):
            i+=1
            tmp = tmp + cEPS*(1+growth)**i
            if (tmp>price):
                break
        return i
    
    def get_earningsperShare(self):
        eps = x.get_earnings_per_share()
        return eps
    
    def get_PE(self):
        #print(self._stock_summary_data('trailingPE'))
        #print(self._stock_summary_data('forwardPE'))
        if not(self._stock_summary_data('trailingPE')):
            return self._stock_summary_data('forwardPE')
        if not(self._stock_summary_data('forwardPE')):
            return self._stock_summary_data('trailingPE')
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        #print('suggested price:', suggestPrice)
        #print('stock price:', stockprice)
        if isinstance(suggestPrice, str):
            return 'skip due to negative eps'
        elif suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'     

In [500]:
class batch_process:
    def __init__(self, tickers, section):
        self.tickers = tickers
        self.jsfile = section
        with open(section, "w") as f:
            s = {"data":[]}
            js.dump(s, f, indent = 4)
    def batch_run(self):       
        for i in range(np.size(self.tickers)):
            try:
                print(self.tickers[i])
                x = cookFinancials(self.tickers[i])
                bv = x.get_BV(20)
                bv.insert(0, x.get_book_value())
                print(bv)
                bvgr = x.get_BV_GR_median(bv)
                print(bvgr)
                growth = bvgr[1]
                cEPS = x.get_earningsperShare()
                years = 3;
                rRate = 0.25;
                safty = 0.5
                PE = x.get_PE()
                price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
                print(price)
                stickerPrice = x.get_current_price()
                decision = x.get_decision(price[1],stickerPrice)
                print(decision)
                y2pb = 0
                roic = 0
                mcap = 0
                cashflow = 0
                priceSales = 0
                if decision == 'strong buy':
                    y2pb = x.payBackTime(stickerPrice, cEPS, growth)
                    roic = x.get_ROIC()
                    mcap = x.get_marketCap_B()
                    cashflow = (x.get_totalCashFromOperatingActivities())
                    priceSales = x.get_pricetoSales()               
                s = {
                    self.tickers[i]:{
                        "decision":decision,
                        "suggested price":price[1],
                        "stock price":x.get_current_price(),                     
                        "Payback years": y2pb,
                        "Book Value": bv,
                        "ROIC": roic,
                        "market cap (b)": mcap,
                        "cashflow": cashflow,
                        "priceSalesRatio":priceSales,
                        "PE": PE
                    }
                }
                print(s)
                with open(self.jsfile, "r") as f:
                    data = js.load(f)
                    cont = data['data']
                    cont.append(s)
                with open(self.jsfile, "w") as f:
                    js.dump(data, f, indent=4) 
                print('=====================================')
            except Exception:
                pass

In [501]:
x = cookFinancials('EGHT')

In [502]:
bv = x.get_BV(20)
bv.insert(0, x.get_book_value())
print(bv)
bvgr = x.get_BV_GR_median(bv)
print(bvgr)
growth = bvgr[1]
cEPS = x.get_earningsperShare()
years = 3;
rRate = 0.25;
safty = 0.5
PE = x.get_PE()
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
                

[174459000, 190731000, 249390000, 218774000, 288601000]
(4, -0.16026189177715494)


In [503]:
print(price)
stickerPrice = x.get_current_price()
decision = x.get_decision(price[1],stickerPrice)
print(decision)
y2pb = 0
roic = 0
mcap = 0
priceSales = 0

NA
skip due to negative eps


In [504]:
from get_all_tickers import get_tickers as gt
from get_all_tickers.get_tickers import Region

In [505]:
tickers = gt.get_tickers()



In [507]:
filtered_by_sector = gt.get_tickers_filtered(mktcap_min=1e3, sectors=gt.SectorConstants.TECH)

In [296]:
filtered_by_sector

['DDD',
 'EGHT',
 'AER',
 'AL',
 'AYX',
 'AMN',
 'PLAN',
 'ANET',
 'ARW',
 'ASAN',
 'ASX',
 'ASGN',
 'ATHM',
 'AVLR',
 'BILL',
 'BKI',
 'BB',
 'BOX',
 'AI',
 'CACI',
 'CANG',
 'CDAY',
 'CHX',
 'CCC',
 'CLDR',
 'NET',
 'CLGX',
 'CTS',
 'CW',
 'DQ',
 'MSP',
 'DELL',
 'DM',
 'DOV',
 'DXC',
 'DT',
 'ETN',
 'ESTC',
 'DAVA',
 'EPAC',
 'EPAM',
 'EB',
 'EVTC',
 'AQUA',
 'FDS',
 'FSLY',
 'FIT',
 'GTES',
 'GB',
 'GLOB',
 'GDDY',
 'GWRE',
 'HPE',
 'HPQ',
 'HUBS',
 'HUYA',
 'INFO',
 'ITW',
 'INFY',
 'IR',
 'NSP',
 'IBM',
 'IPG',
 'JBL',
 'JKS',
 'JBT',
 'JNPR',
 'KAI',
 'KFY',
 'LDOS',
 'LSPD',
 'LN',
 'LINX',
 'RAMP',
 'MAN',
 'MAXR',
 'MXL',
 'MDLA',
 'MFGP',
 'MODN',
 'MSI',
 'NEWR',
 'NOK',
 'NVT',
 'OMC',
 'OCFT',
 'ORCL',
 'PD',
 'PAGS',
 'PLTR',
 'PANW',
 'PSN',
 'PAYC',
 'PNR',
 'PRSP',
 'PING',
 'PINS',
 'PRG',
 'PRO',
 'PSTG',
 'QTWO',
 'RXN',
 'RNG',
 'RHI',
 'SAIL',
 'CRM',
 'SAP',
 'SAIC',
 'NOW',
 'SHOP',
 'SSTK',
 'WORK',
 'SMAR',
 'SNAP',
 'SNOW',
 'SOGO',
 'SWI',
 'SPXC',
 'FLOW',

In [508]:
y = batch_process(filtered_by_sector, 'tech.json')

In [ ]:
y.batch_run()


DDD
[428518000, 522159000, 578369000, 618854000, 629873000]
(4, -0.08130319860154042)
NA
skip due to negative eps
{'DDD': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 11.55, 'Payback years': 0, 'Book Value': [428518000, 522159000, 578369000, 618854000, 629873000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 46.2}}
EGHT
[174459000, 190731000, 249390000, 218774000, 288601000]
(4, -0.16026189177715494)
NA
skip due to negative eps
{'EGHT': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 34.7, 'Payback years': 0, 'Book Value': [174459000, 190731000, 249390000, 218774000, 288601000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 867.50006}}
AER


In [164]:
t = 'VCEL'

In [165]:
t[1]

'C'